In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import h5py

In [ ]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

from torch.utils.data import TensorDataset, DataLoader

## Import Model

In [ ]:
from FBPConvNet import FBPConvNet

## Import train_net

In [ ]:
# Import train_net
import sys
sys.path.append('../')
from net_utils import train_net

# Load Data

In [ ]:
# set path to data
pathtodata = '../data/LineEllipses20.hdf5'
batch_size = 4

In [ ]:
f = h5py.File(pathtodata,'r')
trainingset = f['training']
training_data = f['training/training_data']
training_labels = f['training/training_labels']

In [ ]:
trainset = TensorDataset(torch.Tensor(training_data[:5000]).unsqueeze(1),
                        torch.Tensor(training_labels[:5000]).unsqueeze(1))
testset = TensorDataset(torch.Tensor(training_data[5000:8000]).unsqueeze(1),
                        torch.Tensor(training_labels[5000:8000]).unsqueeze(1))

In [ ]:
f.close()

In [ ]:
trainloader = DataLoader(trainset, batch_size=batch_size,shuffle=True)
testloader = DataLoader(testset, batch_size=4,shuffle=False)

# Train Net

In [ ]:
num_epochs = 500
GPU = torch.cuda.is_available()
print('Using GPU:',GPU)

# save path for trained models
weightpath = './weights/'

# save every x epochs
save_epoch = 50

In [ ]:
net = FBPConvNet()
if GPU:
    net = net.cuda()

In [ ]:
train_net(net,trainloader,num_epochs,GPU,weightpath=weightpath,
         save_epoch=save_epoch,lr=0.01,momentum=0.99,saveweights=True)

# Test Net
uncomment first cell to load already trained net

In [ ]:
# weightspath = '../epoch_500.weights'
# checkpoint = torch.load(weightspath)
# net.load_state_dict(checkpoint)

In [ ]:
testiter = iter(testloader)

In [ ]:
imgs,labels = testiter.next()
num = imgs.size(0)
if GPU:
    est_imgs = net.forward(Variable(imgs.cuda()))
    est_imgs = est_imgs.cpu().data
else:
    est_imgs = net.forward(Variable(imgs))

plt.figure(figsize=(12,4*num))
for i in range(num):
    mse =torch.mean((imgs[i,0,:,:]-labels[i,0,:,:])**2)
    plt.subplot(num,3,3*i+1)
    plt.imshow(imgs[i,0,:,:].numpy())
    plt.title('Noisy %i, MSE=%.4f' % (i,mse))
    plt.axis('off')
    
    mse =torch.mean((est_imgs[i,0,:,:]-labels[i,0,:,:])**2)
    
    plt.subplot(num,3,3*i+2)
    plt.imshow(est_imgs[i,0,:,:].numpy())
    plt.title('Pred %i, MSE = %.4f' % (i, mse))
    plt.axis('off')
    
    plt.subplot(num,3,3*i+3)
    plt.imshow(labels[i,0,:,:].numpy())
    plt.title('Ground Truth %i' % (i))
    plt.axis('off')
